In [0]:
import datetime
from datetime import date, datetime
from pyspark.sql import SparkSession

In [0]:
dbutils.widgets.text(name='fec_ejecucion', defaultValue='', label='fecha de ejecucion')
txt_fec_ejecucion = dbutils.widgets.get(name='fec_ejecucion')

In [0]:
today = datetime.strptime(txt_fec_ejecucion, "%Y-%m-%d")
year = str(today.year)
month = str(today.month).zfill(2)
day = str(today.day).zfill(2)

In [0]:
ruta_silver = "../SILVER/"
ruta_gold = "../GOLD/"

In [0]:
def log_monitoring(notebook_destino, start_process_time, end_process_time, elapsed_seconds_time, error_message, tabla_destino, step_extract_records, step_load_records, step_stage_zone):
    monitoring = spark.createDataFrame([( 
                                   notebook_destino, 
                                   start_process_time, 
                                   end_process_time,
                                   elapsed_seconds_time,
                                   error_message,
                                   step_extract_records,
                                   step_load_records,
                                   step_stage_zone
                                   )],
                                        ["entity_name", 
                                         "start_process_time", 
                                         "end_process_time", 
                                         "elapsed_seconds_time", 
                                         "error_message",
                                         "step_extract_records",
                                         "step_load_records",
                                         "step_stage_zone"
                                         ])
    monitoring.write.format("delta").mode("append").option("path", "dbfs:/mnt/lakehouse-silver/log/"+tabla_destino+"/monitoring.delta").saveAsTable("monitoring."+step_stage_zone+"_"+tabla_destino)

In [0]:
def notebook_run (step_stage_zone, notebook_destino, tabla_destino):
    start_process = datetime.now()
    start_process_time = start_process.strftime("%Y-%m-%d %H:%M:%S")
    error_message = ""
    try:
        response = dbutils.notebook.run(notebook_destino, 3600, {"fec_ejecucion":txt_fec_ejecucion})
        step_extract_records = eval(response)[0]
        step_load_records = eval(response)[1]
    except Exception as e:
        error_message = "Error al ejecutar con mensaje: "+ str(e)
        print("Error en: " + notebook_destino)
        step_extract_records = 0
        step_load_records = 0
    end_process = datetime.now()
    end_process_time = end_process.strftime("%Y-%m-%d %H:%M:%S")
    dtime = end_process - start_process
    elapsed_seconds_time = str(int(dtime.total_seconds()))
    log_monitoring(notebook_destino, start_process_time, end_process_time, elapsed_seconds_time, error_message, tabla_destino, step_extract_records, step_load_records, step_stage_zone)

In [0]:
l_notebook_tbl = [
                    ["silver", ruta_silver+"silver_colegio", "MST_ADM_COLEGIO"], 
                    ["silver", ruta_silver+"silver_alumno", "MST_TRV_ALUMNO"],
                    ["silver", ruta_silver+"silver_campus", "MST_TRV_CAMPUS"],
                    ["silver", ruta_silver+"silver_carrera", "MST_TRV_CARRERA"],
                    ["silver", ruta_silver+"silver_ciudad", "MST_TRV_GEO_CIUDAD"],
                    ["silver", ruta_silver+"silver_comuna", "MST_TRV_GEO_COMUNA"],
                    ["silver", ruta_silver+"silver_curriculum", "MST_TRV_CURRICULUM"],
                    ["silver", ruta_silver+"silver_etnia", "MST_TRV_ETNIA"],
                    ["silver", ruta_silver+"silver_facultad", "MST_TRV_FACULTAD"],
                    ["silver", ruta_silver+"silver_ficha_alumno", "ADM_FICHA_ALUMNO"],
                    ["silver", ruta_silver+"silver_hist_carrera", "HIST_CARRERA"],
                    ["silver", ruta_silver+"silver_pais", "MST_TRV_GEO_PAIS"],
                    ["silver", ruta_silver+"silver_param", "MST_TRV_PARAM"],
                    ["silver", ruta_silver+"silver_persona", "MST_TRV_PERSONA"],
                    ["silver", ruta_silver+"silver_region", "MST_TRV_GEO_REGION"],
                    ["silver", ruta_silver+"silver_solic_ingr", "ADM_SOLIC_INGR"],
                    ["silver", ruta_silver+"silver_unidad_academica", "MST_TRV_UNIDAD_ACADEMICA"],
                    ["silver", ruta_silver+"silver_postulante", "ADM_POSTULANTE"],
                    ["gold", ruta_gold+"gold_lk_campus", "LK_CAMPUS"],
                    ["gold", ruta_gold+"gold_lk_carrera", "LK_CARRERA"],
                    ["gold", ruta_gold+"gold_lk_colegio", "LK_COLEGIO"],
                    ["gold", ruta_gold+"gold_lk_curriculum", "LK_CURRICULUM"],
                    ["gold", ruta_gold+"gold_lk_facultad", "LK_FACULTAD"],
                    ["gold", ruta_gold+"gold_lk_persona", "LK_PERSONA"],
                    ["gold", ruta_gold+"gold_lk_via_admision", "LK_VIA_ADMISION"],
                    ["gold", ruta_gold+"gold_lk_detalle_via_admision", "LK_DETALLE_VIA_ADMISION"],
                    ["gold", ruta_gold+"gold_ft_postulante", "FT_POSTULANTE"],
                    ["gold", ruta_gold+"gold_ft_admision", "FT_ADMISION"],
                    ["gold", ruta_gold+"gold_ft_hist_carrera", "FT_HIST_CARRERA"],
                    ["gold", ruta_gold+"gold_ft_solic_ingr", "FT_SOLIC_INGR"]
                ]

In [0]:
for d_destino in l_notebook_tbl:
    notebook_run(d_destino[0], d_destino[1], d_destino[2])